<a href="https://colab.research.google.com/github/vktiwary007/stock-move-eureka/blob/main/stockMovement3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [252]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.4.1
Eager mode:  True
Hub version:  0.11.0
GPU is available


In [253]:
import io
#df = pd.read_csv('Apple60.csv', skiprows=0)

from sklearn.model_selection import train_test_split
df = pd.read_csv('Para_News15_Combined15.csv', sep=',', header=0)
df["Movement"].replace({-1: 0}, inplace=True)
#data = df.loc[df['Stock'] == 'Apple']
data = df
#train_df = data
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [254]:
data[:10]

,Source,DateTime,News,Movement,Stock
0,www.businesswire.com,2017-12-07 20:00:00+00:00,NAPE Summit Week will also feature the annual ...,1,Apple
1,seekingalpha.com,2017-12-18 20:05:00+00:00,"With such a large revenue pie, naturally we be...",1,Apple
2,www.thestreet.com,2017-12-26 13:48:00+00:00,1 . -- U.S. stock futures turned lower on Tues...,1,Apple
3,www.thestreet.com,2017-12-27 13:35:00+00:00,1 . -- U.S. stock futures traded slightly high...,0,Apple
4,www.thestreet.com,2017-12-28 13:13:00+00:00,2. Here Comes the Data The economic calendar i...,0,Apple
5,www.thestreet.com,2017-12-28 13:13:00+00:00,"Apple CEO Tim Cook Gets a Big Raise, but He'll...",1,Amazon
6,www.thestreet.com,2017-12-29 13:40:00+00:00,1. -- The Dow Sets Yet Another Record U.S. sto...,0,Apple
7,www.thestreet.com,2017-12-29 13:48:00+00:00,Cramer credited his daughters with turning him...,0,Apple
8,www.thestreet.com,2017-12-29 13:48:00+00:00,For those with a little more time and know-how...,0,Amazon
9,www.proactiveinvestors.co.uk,2018-01-02 13:01:00+00:00,Here are the stocks that are in play in pre-ma...,1,Apple


In [255]:
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from tqdm import tqdm
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

MAX_NB_WORDS = 40000
max_seq_len = 1000

raw_docs_train = train_df['News'].tolist()
#raw_docs_test = test_df['News'].tolist()

print(raw_docs_train[:10])

num_classes = 2
processed_docs_train = []

stop_words = set(stopwords.words('english'))

for doc in tqdm(raw_docs_train):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words]
  processed_docs_train.append(" ".join(filtered))




  0%|          | 61/20479 [00:00<00:42, 481.02it/s]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['MARKET COMFORT Loss-making Spotify, which has prioritized rapid growth over profit and whose closest rival is Apple Inc’s ( AAPL.O ) Apple Music, launched in 2008 and had 71 million subscribers at the end of 2017.', 'A key question heading into Wednesday is whether the flagging info tech sector can turn things around. Tuesday’s steep losses wiped out the sector’s 2018 gains and appeared to spread misery across Wall Street. Maybe a dose of positive data might help. The government said in its final estimate that Q4 GDP rose 2.9%. That was well above its prior estimate of 2.5%, and also above Wall Street analysts’ consensus of 2.6%. If you round up, that means growth continues to be near the 3% level, a definite improvement from the recent past. However, many a

100%|██████████| 20479/20479 [00:32<00:00, 627.56it/s]


In [256]:
raw_docs_test = test_df['News'].tolist()
processed_docs_test = []

for doc in tqdm(raw_docs_test):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words]
  processed_docs_test.append(" ".join(filtered))

100%|██████████| 5120/5120 [00:08<00:00, 635.48it/s]


In [257]:
test_data = pd.read_csv('Amazon60_Major.csv', sep=',', header=0)

test_corpus = test_data['News'].tolist()

processed_test_sample = []
for doc in tqdm(test_corpus):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words]
  processed_test_sample.append(" ".join(filtered))

100%|██████████| 229/229 [00:00<00:00, 818.25it/s]


In [258]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  
#tokenizer.fit_on_texts(processed_docs_train) 
vocab_length = len(tokenizer.word_index) + 1

print(vocab_length)

46647


In [259]:
MAX_NB_WORDS = vocab_length
max_seq_len = 1000

word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_index = tokenizer.word_index

word_seq_train = pad_sequences(word_seq_train, maxlen=max_seq_len)

print(len(word_index))

46646


In [260]:
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)

word_seq_test = pad_sequences(word_seq_test, maxlen=max_seq_len)


In [261]:
word_seq_testsample = tokenizer.texts_to_sequences(processed_test_sample)

word_seq_testsample = pad_sequences(word_seq_testsample, maxlen=max_seq_len)

In [74]:
import requests, zipfile, io
zip_file_url = "https://nlp.stanford.edu/data/glove.twitter.27B.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

print("GloVE available locally")

KeyboardInterrupt: ignored

In [262]:
import codecs

embeddings_index = {}
#f = codecs.open(‘crawl-300d-2M.vec’, encoding=’utf-8')
# for Glove
f = codecs.open('glove.twitter.27B.100d.txt', encoding='utf-8')
for line in tqdm(f):
  values = line.rstrip().rsplit(' ')
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

1193515it [00:55, 21673.37it/s]


In [263]:
words_not_found = []
embed_dim = 100
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
  if i >= nb_words:
     continue
  embedding_vector = embeddings_index.get(word)
  
  if (embedding_vector is not None) and len(embedding_vector) > 0:
     embedding_matrix[i] = embedding_vector
  else:
     words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

number of null word embeddings: 16840


In [264]:
from keras.layers import BatchNormalization
from keras.layers import Embedding, Bidirectional, Dense, Dropout, LSTM, Flatten, Conv1D, MaxPooling1D, Activation
from keras.initializers import Constant

model = tf.keras.Sequential()
model.add(Embedding(nb_words, embed_dim, input_length=max_seq_len, weights=[embedding_matrix],trainable=False))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
#model.add(Flatten())
#model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 1000, 100)         4664700   
_________________________________________________________________
dropout_17 (Dropout)         (None, 1000, 100)         0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 996, 64)           32064     
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 249, 64)           0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 100)               66000     
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 101       
Total params: 4,762,865
Trainable params: 98,165
Non-trainable params: 4,664,700
______________________________________

In [265]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import to_categorical

labels = to_categorical(np.asarray(train_df['Movement']))
y_train = np.array(train_df['Movement'])

print(labels[:10])
print(y_train[:10])


[[0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]
[1 1 0 1 0 0 0 1 0 0]


In [266]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [267]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_m, precision_m, recall_m])
es_callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(word_seq_train, y_train, epochs=15)

Epoch 1/15
640/640 [==============================] - 13s 18ms/step - loss: 0.6979 - accuracy: 0.4989 - f1_m: 0.3269 - precision_m: 0.3725 - recall_m: 0.4040
Epoch 2/15
640/640 [==============================] - 11s 18ms/step - loss: 0.6898 - accuracy: 0.5295 - f1_m: 0.4395 - precision_m: 0.5520 - recall_m: 0.4544
Epoch 3/15
640/640 [==============================] - 11s 18ms/step - loss: 0.6799 - accuracy: 0.5671 - f1_m: 0.5132 - precision_m: 0.5786 - recall_m: 0.5358
Epoch 4/15
640/640 [==============================] - 11s 18ms/step - loss: 0.6596 - accuracy: 0.6024 - f1_m: 0.5613 - precision_m: 0.6097 - recall_m: 0.5517
Epoch 5/15
640/640 [==============================] - 11s 18ms/step - loss: 0.6326 - accuracy: 0.6334 - f1_m: 0.6004 - precision_m: 0.6329 - recall_m: 0.5947
Epoch 6/15
640/640 [==============================] - 11s 18ms/step - loss: 0.6107 - accuracy: 0.6571 - f1_m: 0.6237 - precision_m: 0.6573 - recall_m: 0.6190
Epoch 7/15
640/640 [==============================] 

In [268]:
y_test1 = np.array(test_df['Movement'])
y_test2 = to_categorical(np.asarray(test_df['Movement']))
probas = model.predict(word_seq_test)
predict_labels = (probas < 0.5).astype(np.int)

test_df["Movement"].replace({0: -1}, inplace=True)
#test_df['Predicted'] = predict_labels
predict_labels = pd.DataFrame(predict_labels)
predict_labels.replace({0: -1}, inplace=True)
predict_labels.to_csv('test_label.csv')
test_df.to_csv('test_news.csv')

loss, accuracy, f1_score, precision, recall = model.evaluate(word_seq_test, y_test1, verbose=0)
print('Accuracy: %f' % (accuracy*100))
print('loss: %f' % (loss))
print('f1: %f' % (f1_score))
print('precision: %f' % (precision))
print('recall: %f' % (recall))

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


Accuracy: 54.414064
loss: 0.820437
f1: 0.488266
precision: 0.541276
recall: 0.460953


In [32]:
test_data["Movement"].replace({-1: 0}, inplace=True)
y_testsample = np.array(test_data['Movement'])
probas = model.predict(word_seq_testsample)
predict_labels = (probas < 0.5).astype(np.int)

print(predict_labels)
np.savetxt("foo.csv", predict_labels, delimiter=",")
loss, accuracy, f1_score, precision, recall = model.evaluate(word_seq_testsample, y_testsample, verbose=0)
print('Accuracy: %f' % (accuracy*100))
print('loss: %f' % (loss*100))
print('f1: %f' % (f1_score))
print('precision: %f' % (precision))
print('recall: %f' % (recall))

[[0]
 [1]
 [0]
 ...
 [0]
 [1]
 [0]]
Accuracy: 67.945069
loss: 59.957469
f1: 0.650129
precision: 0.656496
recall: 0.694183
